In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json

import getpass
from openai import OpenAI

import pandas as pd
import numpy as np
from tqdm import tqdm
import time

import sqlalchemy as sa
from sqlalchemy import create_engine

In [2]:
page = requests.get('https://en.wikipedia.org/wiki/Wikipedia:Good_articles/Video_games')
soup = BeautifulSoup(page.content, 'html.parser')
links = soup.find_all('a')
urls = [link.get('href') for link in links if link.get('href') is not None]

In [18]:
excluded_patterns = [
    '#',  # Anchor links
    '/wiki/Main_Page',
    '/wiki/Wikipedia:',
    '/wiki/Portal:',
    '/wiki/Special:',
    '/wiki/Help:',
    '//en.wikipedia.org/wiki/Wikipedia:',
    'https://donate.wikimedia.org/wiki/Special:',
    '/w/index.php?title=Special:',
    '/wiki/Special:My',
    'https://www.wikidata.org/wiki/Special:',
    '/w/index.php?title=',
    '/wiki/File:',
    '/wiki/Category:',
    '/wiki/Template:',
    '/wiki/Wikipedia_talk:',
    '/wiki/User:',
]

filtered_urls = ['https://en.wikipedia.org' + url for url in urls if not any(pattern in url for pattern in excluded_patterns)]

In [3]:
username = 'admin'
passkey = getpass.getpass('Enter your SingleStore admin password: ')
connection_string = 'svc-4d9215ce-fc86-42cb-9bf7-c5e1383bb647-dml.aws-oregon-4.svc.singlestore.com'
port = '3306'
database = 'wikipedia_test'

In [4]:
engine = create_engine(f'mysql+pymysql://{username}:{passkey}@{connection_string}:{port}/{database}')
conn = engine.connect()
print('Connected to SingleStore')

Connected to SingleStore


In [5]:
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

API_KEY = getpass.getpass('OpenAI API Key: ')
client = OpenAI(api_key=API_KEY)

In [9]:
def create_table():
    '''Creates the table in SingleStore'''
    conn.execute(sa.text('''DROP TABLE IF EXISTS wiki_scrape;'''))
    conn.execute(sa.text('''
    CREATE TABLE wiki_scrape(
        id BIGINT AUTO_INCREMENT PRIMARY KEY,
        url VARCHAR(255),
        paragraph TEXT,
        embedding VECTOR(1536, F32) NOT NULL,
        FULLTEXT (paragraph),
        VECTOR INDEX (embedding) INDEX_OPTIONS '{"index_type":"IVF_PQ"}'
    );
    '''))
    print('Table created')

def clear_table():
    '''Clears the table of all data'''
    conn.execute(sa.text('TRUNCATE TABLE wiki_scrape;'))
    print('Table cleared of all data')

In [21]:
clear_table()

Table cleared of all data


In [6]:
def clean_text(text):
    '''cleans the text of a wiki page'''
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\<.*?\>', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\t', '', text)
    text = re.sub(r'\s\s+', ' ', text)
    return text

def get_text(url):
    '''Gets the text from a wiki page and returns it as a string.'''
    try:
        page = requests.get(url)
        page.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code
        soup = BeautifulSoup(page.content, 'html.parser')
        paragraphs = soup.find_all('p')
        cleaned_paragraphs = [clean_text(p.text) for p in paragraphs if p.text.strip()]
        return cleaned_paragraphs
    except requests.RequestException as e:
        # print(f"Error fetching URL {url}: {e}")
        return []

def get_embedding(text, model=EMBEDDING_MODEL):
    '''Generates the OpenAI embedding from an input `text`.'''
    try:
        if isinstance(text, str):
            response = client.embeddings.create(input=[text], model=model)
            embedding = response.data[0].embedding
            # return np.array(embedding).tobytes()
            return json.dumps(embedding)
        else:
            # print(f"Invalid input: {text}")
            return None
    except Exception as e:
        # print(f"Error generating embedding: {e}")
        return None

def text_embedding_df(url):
    '''Creates a dataframe of the text from a wiki page and the OpenAI embeddings of that text'''
    text = get_text(url)
    embeddings = [get_embedding(t) for t in text]
    df = pd.DataFrame({'paragraph': text, 'embedding': embeddings})
    return df

def scrape_wiki(url_list, table_name, engine):
    '''Pushes a dataframe to a SingleStore table'''
    for url in tqdm(url_list):
        dataframe = text_embedding_df(url)
        dataframe['url'] = url 
        dataframe = dataframe[['url', 'paragraph', 'embedding']]
        dataframe = dataframe[dataframe['embedding'].notna()]
        dataframe.to_sql(table_name, con=engine, if_exists='append', index=False)

In [14]:
def search_wiki_page(query, limit=5):
    '''Returns a df of the top k matches to the query ordered by similarity.'''
    query_embedding_vec = get_embedding(query)
    # cast query embedding to vector data type :> vector(1536) - returning json dumps in embedding
    statement = sa.text(
        f'''SELECT paragraph, v <*> :query_embedding :> vector(1536) AS similarity
        FROM vecs
        ORDER BY similarity USE INDEX (ivfpq) DESC
        LIMIT :limit;'''
    )
    print("Searching for matches...")
    start_time = time.time()
    results = conn.execute(statement, {"query_embedding": query_embedding_vec, "limit": limit})
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Search complete in {execution_time} seconds.")
    results_as_dict = results.fetchall()
    return results_as_dict

def ask_wiki_page(query, limit=5, temp=0.0):
    '''Uses a RAG to answer a question from the wiki page'''
    results = search_wiki_page(query, limit)
    print("Asking Chatbot...")
    prompt = f'''Excerpt from the conversation history: 
        {results}
        Question: {query}
        
        Based on the conversation history, try to provide the most accurate answer to the question. 
        Consider the details mentioned in the conversation history to formulate a response that is as 
        helpful and precise as possible. please provide links to WIKIPEDIA ARTICLES TO LOOK AT FOR MORE INFORMATION.

        Most importantly, IF THE INFORMATION IS NOT PRESENT IN THE CONVERSATION HISTORY, PLEASE DO NOT MAKE UP AN ANSWER.'''
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who is answering questions about an article."},
            {"role": "user", "content": prompt}
        ],
        temperature=temp
    )
    response_message = response.choices[0].message.content
    return response_message

In [16]:
ask_wiki_page("Tell me about the Mario video game franchise and its history. What is it known for?", limit=15)

Searching for matches...
Search complete in 0.19211912155151367 seconds.
Asking Chatbot...


"The Mario video game franchise is known for its platform games featuring the iconic character Mario. It was developed and published by Nintendo, starting with the arcade game Mario Bros. in 1983. The franchise was designed by Shigeru Miyamoto and Gunpei Yokoi. Mario Bros. was followed by Super Mario RPG, which was critically acclaimed for its humor and 3D-rendered graphics. This game inspired Nintendo's other role-playing series, Paper Mario and Mario & Luigi. Another notable game in the franchise is Superstar Saga, developed by AlphaDream and directed by Yoshihiko Maekawa. It features the voice of Charles Martinet, who is the official voice of Mario in Nintendo's Mario franchise. The franchise also includes games like Super Mario Land, Hotel Mario, and Super Mario Kart. For more detailed information, you can refer to the Wikipedia articles on the Mario franchise (https://en.wikipedia.org/wiki/Mario) and its individual games."